In [7]:
# @title Parameters
LOCATION = "us-central1" #@param {type:"string"}
#PROJECT_NUMBER = 
#PROJECT_ID = "" #@param {type:"string"}

PROJECT_ID = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_ID)"

if len(PROJECT_ID) > 1:
    PROJECT_ID = PROJECT_ID[1]
else:
    PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

PROJECT_NUMBER = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
if len(PROJECT_NUMBER) > 1:
    PROJECT_NUMBER = PROJECT_NUMBER[1]
else:
    PROJECT_NUMBER = PROJECT_NUMBER[0]

print(PROJECT_NUMBER)

TOKEN_ENDPOINT_URI = 'https://www.googleapis.com/oauth2/v3/tokeninfo?access_token='

IMAGEGEN_MODEL = "imagegeneration@002" #models available: imagegeneration@001 and imagegeneration@002
ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGEGEN_MODEL}'

IMAGE_TEXT_MODEL = "imagetext"
IMAGEN_ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGE_TEXT_MODEL}'

RATE_LIMIT = 10 #default quota 10 Online prediction requests per base model per minute per region per base_model

vertexai_text_bison ="text-bison@001"


import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

ml-demo-384110
1008225662928


In [8]:
import logging
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)


In [22]:
from IPython.display import display, Markdown, Latex
display(Markdown('*some markdown* $\phi$'))


*some markdown* $\phi$

## Generate property description based on input text

In [10]:
import pandas as pd

from vertexai.preview.language_models import TextGenerationModel

generation_model = TextGenerationModel.from_pretrained(vertexai_text_bison)

def llm(prompt,         max_output_tokens=1024,
        temperature=0,
        top_p=0.1,
        top_k=1,        
):
    result = generation_model.predict(
        prompt=prompt,
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,        
    )
    return result

In [26]:
q = "Facts: 2000 square feet, Austin, TX, 4 bed, 3 bath, $1,200,000, 78702"

example= """Facts: 3065 square feet, Austin, TX, 4 bed, 4 bath, 1,250,000, 78750
** Description ** 
Completely remodeled and added over 1000sf and a modern spa and pool. 
2 screened porches, engineering wood flooring, all new windows and the \"gem\" of the house is the large kitchen island.
All expanded areas were added with foam insulation, new HVAC -2020. 3 bedrooms on the main floor with 1 having a private full bathroom.  
The primary bedroom is located on the lower pool level with a sitting area and fireplace down the hall to the master.  
Beautiful primary bathroom and large closet.  
** Localisation ** 
This home sits on over 1/3 acre and all outside wood decking is made with synthetic long-lasting wood.  
This home''s design is well thought out with convenience and a touch of modern elegance!  Great Schools!
         """

prompt = "Context: You are a real estate agent that needs to write a property description based on facts about the property. "+ example + " Q:"+ q +" A: "

display(llm(
    prompt=prompt,
    max_output_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    top_k=40)
)


** Description ** 
Welcome to this beautiful 4 bedroom 3 bath home in the heart of Austin! This home has been completely remodeled with new flooring, paint, and fixtures. The kitchen has been updated with granite countertops and stainless steel appliances. The master bedroom has a large walk-in closet and an en suite bathroom. The backyard is private and has a patio and a covered porch. This home is located close to schools, shopping, and restaurants.
** Localisation ** 
This home is located in the heart of Austin, Texas. The neighborhood is quiet and family-friendly. The home is close to schools, shopping, and restaurants.

**Description**
Completely remodeled and added over 1000sf and a modern spa and pool. 
2 screened porches, engineering wood flooring, all new windows and the \"gem\" of the house is the large kitchen island.
All expanded areas were added with foam insulation, new HVAC -2020. 3 bedrooms on the main floor with 1 having a private full bathroom.  
The primary bedroom is located on the lower pool level with a sitting area and fireplace down the hall to the master.  
Beautiful primary bathroom and large closet.  
** Localisation ** 
This home sits on over 1/3 acre and all outside wood decking is made with synthetic long-lasting wood.  
This home''s design is well thought out with convenience and a touch of modern elegance!  Great Schools!

## Generate property description based on enriched data (googlemap)

In [9]:
from google.cloud import secretmanager
import google_crc32c


def access_secret_version(
    project_id: str, secret_id: str, version_id: str
) -> str:
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    VERBOSE = False
    
    # Import the Secret Manager client library.
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    # Verify payload checksum.
    crc32c = google_crc32c.Checksum()
    crc32c.update(response.payload.data)
    if response.payload.data_crc32c != int(crc32c.hexdigest(), 16):
        print("Data corruption detected.")
        return response

    # Print the secret payload.
    #
    # WARNING: Do not print the secret in a production environment - this
    # snippet is showing how to access the secret material.
    payload = response.payload.data.decode("UTF-8")
    if VERBOSE == True:
        print(f"Plaintext: {payload}")
    
    return payload

import os
CSE_ID = access_secret_version(PROJECT_NUMBER, "CSE_ID", "latest")
GOOGLE_API_KEY = access_secret_version(PROJECT_NUMBER, "GOOGLE_API_KEY", "latest")
os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY
os.environ["GPLACES_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = CSE_ID



In [ ]:

"""
Maison Saint Laurent De La Mer 7 pièces. Opportunité !
Venez découvrir cette belle maison en plein bourg de Saint-Laurent-De-La-Mer avec vue sur mer.
Au rez-de-chaussée vous trouverez une entrée desservant un double garage, cuisine équipée & aménagée, salle d'eau, WC, ainsi que deux chambres.
Le premier étage vous offrira un palier distribuant un séjour avec coin cuisine entièrement équipé & aménagé, salle de bains avec WC ainsi que deux chambres dont une avec salle d'eau attenante.
Au dernier étage, un palier offrant deux chambres, placards & combles.
Les atouts de ce bien : le jardin clôturé & arboré ainsi que son accès plage en 5 minutes à pieds. 
"""



In [33]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType
import os
from datetime import datetime

from langchain.llms import VertexAI
from langchain.callbacks import StdOutCallbackHandler

In [34]:
from langchain.tools import GooglePlacesTool

places = GooglePlacesTool()

In [40]:

callbacks = [StdOutCallbackHandler()]
llm = VertexAI( callbacks=callbacks)

tools_search = load_tools(["google-search"], llm=llm, callbacks=callbacks)
tools_wikipedia = load_tools(["wikipedia"], llm=llm, callbacks=callbacks)

tools_std = load_tools(["google-search",  "wikipedia"], llm=llm, callbacks=callbacks)



tools = [places]+tools_wikipedia

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
property_location = "Saint Malo France"
prompt ="Context: You are a real estate agent that needs to write a property description based on facts about the property. "+ example + " LOCATION: "+ property_location + " Q:" + q +" A: "

agent.run(
prompt
)



Observation: Page: Mothership Space Net Penthouse
Summary: The Mothership Space Net Penthouse is an aerial hammock in Moab, Utah, hand-woven from 14,000 feet of cordage and rigged by the Desert Rats, a group of more than 50 people spearheaded by slackliner Andy Lewis, during Thanksgiving 2014.The net is suspended over 400 feet above the ground, in a pentagon shape covering approximately 2000 square feet, with five lines going out to the surrounding cliffs. It took Lewis several years to acquire all of the proper materials to build it.  Highliners walk across the five different legs of the net (which are as long as 80 meters or 262 feet), and BASE jumpers leap from the human-sized hole in the middle of the net.  The pentagonal web is over 200 feet away from the nearest cliff.

Page: Michael G. Moye
Summary: Michael George Moye (born August 11, 1954) is an American photographer and a former television writer and producer. In his television career he has written for shows such as The Jef

'Agent stopped due to iteration limit or time limit.'

In [43]:
property_location = "Saint Laurent De La Mer France"
q = """House Saint Laurent De La Mer 7 rooms. Opportunity !
Come and discover this beautiful house in the village of Saint-Laurent-De-La-Mer with sea view.
On the ground floor you will find an entrance leading to a double garage, fitted & equipped kitchen, shower room, WC, as well as two bedrooms.
The first floor will offer you a landing distributing a living room with fully equipped & fitted kitchen area, bathroom with WC as well as two bedrooms, one of which has an adjoining shower room.
On the top floor, a landing offering two bedrooms, cupboards & attic space.
The assets of this property: the fenced & tree-lined garden as well as its beach access in 5 minutes on foot. """

In [ ]:
property_location = "Saint Malo France"
prompt ="Context: You are a real estate agent that needs to write a property description based on facts about the property. " + example  \
+ " LOCATION: "+ property_location + " Q:" + q +" A: "

agent.run(
prompt
)
